# Natural Language Interaction with Data
This notebook shows an example of Natural Language Interaction with Data.

- First a keyspace and table are set up for Credit Card Transactions
- Secondly a prompt will be defined that is based on the [SQL-PaLM](https://arxiv.org/abs/2306.00739) paper.
- This prompt instructs to create CQL code that allows for queries on the data

## Install dependencies

In [ ]:
!pip install ragstack-ai datasets pipdeptree ollama

## Visualize Ragstack dependencies
RAGStack is a curated stack of the best open-source software for easing implementation of the RAG pattern in production-ready applications using DataStax Enterprise, Astra Vector DB or Apache Cassandra as a vector store.

A single command (pip install ragstack-ai) unlocks all the open-source packages required to build production-ready RAG applications with LangChain and DataStax Enterprise, Astra Vector DB or Apache Cassandra.

For each open-source project included in RAGStack, we select a version lineup and then test the combination for compatibility, performance, and security. Our extensive test suite ensures that RAGStack components work well together so you can confidently deploy them in production. We also run security scans on all components using industry-standard tools to ensure that you are not exposed to known vulnerabilities.

In [ ]:
!pipdeptree -p ragstack-ai

## Keeping data locally and within the enterprise firewall
In this notebook we'll keep the data local to ensure maximum safety:

- For the Vector Database, [DataStax Enterprise 7](https://www.datastax.com/blog/get-started-with-the-datastax-enterprise-7-0-developer-vector-search-preview) will be used.
- For the Foundational Model we'll be using [OpenAI's GPT 3.5](https://api.openai.com).

## Get an inference engine with OpenAI started
In case you can not run the inference engine locally, we'll opt for OpenAI's gpt-3.5-turb.

In [ ]:
# Calling OpenAI directly (not ideal because of API and implementation specifics and no support through RAGStack)
# This step is optional and for informational purposes only

from openai import OpenAI
import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key: ")

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {'role': 'system', 'content': "You're a helpful assistant"},
        {'role': 'user', 'content': 'Why is the sky blue?'}
    ]
)

print(response.choices[0].message.content)

In [ ]:
# Calling OpenAI through RAGStack (THIS IS THE DEFAULT IN THIS NOTEBOOK)
# Way better because RAGStack transparantly takes away implementation specifics and provides a clean interface through LangChain

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

client = ChatOpenAI()

messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content="Why is the sky blue?"),
]

response = client.invoke(messages)

print (response.content)

# Now let's run DSE 7 Vector Store
Make sure you have [Docker](https://www.docker.com/) installed.

Run DSE 7 in any of these two ways from a terminal window:
1. `docker-compose up` (using the docker-compose.yml file in the root of this repository)
2. `docker run -e DS_LICENSE=accept -p 9042:9042 datastax/dse-server:7.0.0-alpha.4`

And then create a default keyspace as follows:

In [ ]:
from cassandra.cluster import Cluster

# Connect to DSE7
cluster = Cluster(["host.docker.internal"])
session = cluster.connect()

def execute_statement(statement: str):
    # This is a simple wrapper around executing CQL statements in our
    # Cassandra cluster, and either raising an error or returning the results
    try:
        rows = session.execute(statement)
        return rows.all()
    except:
        print(f"Query Failed: {statement}")
        raise

# Create the default keyspace
execute_statement("CREATE KEYSPACE IF NOT EXISTS default_keyspace WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1}")

# Create the CC keyspace
execute_statement("CREATE KEYSPACE IF NOT EXISTS creditcardtransactions WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1}")

## Create the schema
This schema allows for storing credit card transactions in a simple way. It also creates an SAI Index on the information column that enables the new Text Analyzer for keyword search.

In [ ]:
create_schema_cql = f"""
CREATE TABLE IF NOT EXISTS creditcardtransactions.transactions (
    credit_card_number text,
    transaction_date date,
    transaction_time time,
    transaction_id uuid,
    merchant text,
    information text,
    amount decimal,
    currency text,
    status text,
    PRIMARY KEY ((credit_card_number), transaction_date, transaction_time, transaction_id)
) WITH CLUSTERING ORDER BY (transaction_date DESC, transaction_time DESC);

CREATE CUSTOM INDEX IF NOT EXISTS transactions_information_idx ON creditcardtransactions.transactions (information) USING 'org.apache.cassandra.index.sai.StorageAttachedIndex' WITH OPTIONS = {{ 'index_analyzer': 'standard'}};
"""

# This parses the text above into executable strings by the driver
for statement in create_schema_cql.split(";"):
    if len(statement.strip()):
        execute_statement(statement.strip())

print("Done")

## Load some data
The following step loads some sample data into the database

In [ ]:
create_data_cql = f"""
INSERT INTO creditcardtransactions.transactions (credit_card_number, transaction_date, transaction_time, transaction_id, merchant, information, amount, currency, status) VALUES ('1234567890123456', '2023-03-01', '08:30:00', uuid(), 'Coffee Shop', 'Latte and croissant', 5.75, 'USD', 'Completed');
INSERT INTO creditcardtransactions.transactions (credit_card_number, transaction_date, transaction_time, transaction_id, merchant, information, amount, currency, status) VALUES ('1234567890123456', '2023-03-02', '12:45:00', uuid(), 'Restaurant', 'Lunch with colleagues', 20.50, 'USD', 'Completed');
INSERT INTO creditcardtransactions.transactions (credit_card_number, transaction_date, transaction_time, transaction_id, merchant, information, amount, currency, status) VALUES ('1234567890123456', '2023-03-03', '18:30:00', uuid(), 'Cinema', 'Movie night - 2 tickets', 15.00, 'USD', 'Completed');
INSERT INTO creditcardtransactions.transactions (credit_card_number, transaction_date, transaction_time, transaction_id, merchant, information, amount, currency, status) VALUES ('1234567890123456', '2023-03-05', '09:00:00', uuid(), 'Gas Station', 'Fuel for the week', 45.00, 'USD', 'Completed');
INSERT INTO creditcardtransactions.transactions (credit_card_number, transaction_date, transaction_time, transaction_id, merchant, information, amount, currency, status) VALUES ('1234567890123456', '2023-03-08', '15:00:00', uuid(), 'Online Store', 'Books and electronics', 120.95, 'USD', 'Completed');
INSERT INTO creditcardtransactions.transactions (credit_card_number, transaction_date, transaction_time, transaction_id, merchant, information, amount, currency, status) VALUES ('1234567890123456', '2023-03-10', '20:00:00', uuid(), 'Italian Restaurant', 'Dinner date', 60.30, 'USD', 'Completed');
INSERT INTO creditcardtransactions.transactions (credit_card_number, transaction_date, transaction_time, transaction_id, merchant, information, amount, currency, status) VALUES ('1234567890123456', '2023-03-15', '14:00:00', uuid(), 'Bookstore', 'Educational books', 35.00, 'USD', 'Completed');
INSERT INTO creditcardtransactions.transactions (credit_card_number, transaction_date, transaction_time, transaction_id, merchant, information, amount, currency, status) VALUES ('1234567890123456', '2023-03-18', '11:00:00', uuid(), 'Grocery Store', 'Weekly groceries', 90.75, 'USD', 'Completed');
INSERT INTO creditcardtransactions.transactions (credit_card_number, transaction_date, transaction_time, transaction_id, merchant, information, amount, currency, status) VALUES ('1234567890123456', '2023-03-20', '07:45:00', uuid(), 'Coffee Shop', 'Morning coffee', 3.50, 'USD', 'Completed');
INSERT INTO creditcardtransactions.transactions (credit_card_number, transaction_date, transaction_time, transaction_id, merchant, information, amount, currency, status) VALUES ('1234567890123456', '2023-03-23', '19:00:00', uuid(), 'Electronics Store', 'New headphones', 75.00, 'USD', 'Completed');
INSERT INTO creditcardtransactions.transactions (credit_card_number, transaction_date, transaction_time, transaction_id, merchant, information, amount, currency, status) VALUES ('1234567890123456', '2023-03-25', '10:30:00', uuid(), 'Local Bakery', 'Weekend breakfast treats', 8.50, 'USD', 'Completed');
INSERT INTO creditcardtransactions.transactions (credit_card_number, transaction_date, transaction_time, transaction_id, merchant, information, amount, currency, status) VALUES ('1234567890123456', '2023-03-26', '15:20:00', uuid(), 'Sporting Goods Store', 'Running shoes', 110.00, 'USD', 'Completed');
INSERT INTO creditcardtransactions.transactions (credit_card_number, transaction_date, transaction_time, transaction_id, merchant, information, amount, currency, status) VALUES ('1234567890123456', '2023-03-27', '20:00:00', uuid(), 'Online Subscription Service', 'Monthly streaming service fee', 12.99, 'USD', 'Completed');
INSERT INTO creditcardtransactions.transactions (credit_card_number, transaction_date, transaction_time, transaction_id, merchant, information, amount, currency, status) VALUES ('1234567890123456', '2023-03-29', '13:00:00', uuid(), 'Pharmacy', 'Medication', 23.45, 'USD', 'Completed');
INSERT INTO creditcardtransactions.transactions (credit_card_number, transaction_date, transaction_time, transaction_id, merchant, information, amount, currency, status) VALUES ('1234567890123456', '2023-03-30', '08:15:00', uuid(), 'Public Transit', 'Monthly bus pass', 45.00, 'USD', 'Completed');
INSERT INTO creditcardtransactions.transactions (credit_card_number, transaction_date, transaction_time, transaction_id, merchant, information, amount, currency, status) VALUES ('1234567890123456', '2023-03-31', '18:30:00', uuid(), 'Concert Venue', 'Concert tickets', 90.00, 'USD', 'Completed');
INSERT INTO creditcardtransactions.transactions (credit_card_number, transaction_date, transaction_time, transaction_id, merchant, information, amount, currency, status) VALUES ('1234567890123456', '2023-04-01', '12:00:00', uuid(), 'Pet Store', 'Pet food and toys', 60.75, 'USD', 'Completed');
INSERT INTO creditcardtransactions.transactions (credit_card_number, transaction_date, transaction_time, transaction_id, merchant, information, amount, currency, status) VALUES ('1234567890123456', '2023-04-02', '14:30:00', uuid(), 'Home Improvement Store', 'Gardening supplies', 35.20, 'USD', 'Completed');
INSERT INTO creditcardtransactions.transactions (credit_card_number, transaction_date, transaction_time, transaction_id, merchant, information, amount, currency, status) VALUES ('1234567890123456', '2023-04-03', '19:45:00', uuid(), 'Diner', 'Dinner with friends', 50.00, 'USD', 'Completed');
INSERT INTO creditcardtransactions.transactions (credit_card_number, transaction_date, transaction_time, transaction_id, merchant, information, amount, currency, status) VALUES ('1234567890123456', '2023-04-04', '17:00:00', uuid(), 'Electronics Store', 'Smartphone case', 25.00, 'USD', 'Completed');
"""

# This parses the text above into executable strings by the driver
for statement in create_data_cql.split(";"):
    if len(statement.strip()):
        execute_statement(statement.strip())

print("Done")

# Let's add some metadata to the table
This metadata helps the LLM to understand which tables to use for querying

In [ ]:
add_comments_cql = f"""
ALTER TABLE creditcardtransactions.transactions WITH comment = 'Credit card transactions. Notable columns are: transaction_date, merchant and amount. Column information holds additional information about the purchase.';
"""

# This parses the text above into executable strings by the driver
for statement in add_comments_cql.split(";"):
    if len(statement.strip()):
        execute_statement(statement.strip())

print("Done")

## Run the Natural Language to CQL conversion through the LLM
The code below runs the following steps:
1. It first creates a prompt that tells LLama2 how to handle user queries and convert them to CQL
2. It then takes the resulting CQL and executes this on DSE 7
3. Finally, the results are returned to the user

In [ ]:
TEXT2CQL_PROMPT = """Convert the question to CQL (Cassandra Query Language) that can retrieve an appropriate answer, 
or answer saying that the data model does not support answering such a question in a performant way.
CQL has been extended for full text queries as follows: SELECT * FROM creditcardtransactions.transactions WHERE information : 'word'. Make sure to use the colon as stated! Do this instead of using CONTAINS or LIKE.

[Schema : values (type)]
{schema}

[Partition Keys]
{partition_keys}

[Clustering Keys]
{clustering_keys}

[Q]
{question}

[CQL]
"""


def generate_schema_partition_clustering_keys(keyspace: str = "default_keyspace") -> (str, str):
    """Generates a TEXT2CQL_PROMPT compatible schema for a keyspace"""
    # Get all table names in our keyspace
    table_names = execute_statement(
        f"SELECT table_name, comment FROM system_schema.tables WHERE keyspace_name = '{keyspace}'"
    )
    tn_str = ", ".join(["'" + tn.table_name + "'" for tn in table_names])

    # Now get all the column names corresponding to those tables
    columns = execute_statement(
        f"SELECT * FROM system_schema.columns WHERE table_name IN ({tn_str}) AND keyspace_name = '{keyspace}' ALLOW FILTERING"
    )

    # Now, we construct our prompt template formatted schema, partition_keys, and clustering keys
    # from the table and column objects returned from the DB
    schema = " | ".join([
        f"{keyspace}.{table.table_name} '{table.comment}' : " + " , ".join([
            f"{col.column_name} ({col.type})"
            for col in columns
            if col.table_name == table.table_name
        ])
        for table in table_names
    ])
    partition_keys = " | ".join([
        f"{keyspace}.{table.table_name} : " + " , ".join([
            col.column_name for col in columns
            if col.table_name == table.table_name
            and col.kind == "partition_key"
        ])
        for table in table_names
    ])
    clustering_keys = " | ".join([
        f"{keyspace}.{table.table_name} : " + " , ".join([
            f"{col.column_name} ({col.clustering_order})" for col in columns
            if col.table_name == table.table_name
            and col.kind == "clustering"
        ])
        for table in table_names
    ])
    return schema, partition_keys, clustering_keys


def execute_query_from_question(question: str, debug_cql: bool = True, debug_prompt: bool = False, return_cql: bool = False):
    """Generates and executes CQL from a user question based on LLM output"""
    # Get all of the variables necessary to fill out the prompt
    schema, partition_keys, clustering_keys = generate_schema_partition_clustering_keys(keyspace="creditcardtransactions")
    prompt = TEXT2CQL_PROMPT.format(
        schema=schema,
        partition_keys=partition_keys,
        clustering_keys=clustering_keys,
        question=question,
    )

    if debug_prompt:
        print(f"Prompting model with:\n{prompt}")

    # Get generated CQL from the LLM
    messages = [
        HumanMessage(content=prompt),
    ]
    completion = client.invoke(messages).content

    if debug_cql:
        print(f"Question: {question}\nGenerated Query: {completion}\n")

    # Need to trim trailing ';' if present to work with cassandra-driver
    if completion.find(";") > -1:
        completion = completion[:completion.find(";")]
    # Need to drop backticks
    completion = completion.replace('`', '')

    results = execute_statement(completion)

    if return_cql:
        return (results, completion)
    else:
        return results

In [ ]:
# Show full prompting trace
execute_query_from_question("What did I purchase on 2023-04-04? My credit card number is 1234567890123456", debug_prompt=True)

In [ ]:
# Show full prompting trace
execute_query_from_question("Where did I purchase coffee? My credit card number is 1234567890123456", debug_prompt=True)

# Now we get a full question-answer chain going!

In [ ]:
ANSWER_PROMPT = """Query:
```
{cql}
```

Output:
```
{results_repr}
```
===

Given the above results from querying the DB, answer the following user question:

{question}
"""


def answer_question(question: str, debug_cql: bool = False, debug_prompt: bool = False) -> str:
    """Conducts a full RAG pipeline where the LLM retrieves relevant information
    and references it to answer the question in natural language.
    """
    # Get necessary fields to fill out prompt
    query_results, cql = execute_query_from_question(
        question=question,
        debug_cql=debug_cql,
        debug_prompt=debug_prompt,
        return_cql=True,
    )
    prompt = ANSWER_PROMPT.format(
        question=question,
        results_repr=str(query_results),
        cql=cql,
    )

    if debug_prompt:
        print(f"Prompting model with:\n{prompt}")

    # Get generated answer from the LLM
    messages = [
        HumanMessage(content=prompt),
    ]
    return client.invoke(messages).content


In [ ]:
# Show full prompting trace
print(
    answer_question("Where did I purchase coffee? My credit card number is 1234567890123456", debug_prompt=True)
)

In [ ]:
# Show full prompting trace
print(
    answer_question("How many coffees did I purchase in March 2023? My credit card number is 1234567890123456", debug_prompt=True)
)

In [ ]:
# Show full prompting trace
print(
    answer_question("How much money did I spend in March 2023? My credit card number is 1234567890123456", debug_prompt=True)
)